In [1]:
import os
import sys
import re
import time
import urllib.request
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

df = pd.read_csv('stock/hybe_stock_data.csv')
headers = {"User-Agent"
    :"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

In [2]:
query = "하이브"
encQuery = urllib.parse.quote(query)
date_list = [i.replace('-', '.') for i in df['Date']] # 형태 변환 (2023-06-01 -> 2023.06.01)
index_list = [i for i in range(0, 40)] # new_df에 행 추가 할 때 index 사용
news_df = pd.DataFrame(columns=['Title', 'Date'])

# 날짜마다 관련도 기준 상위 40개 기사 추출 
for date in tqdm(date_list): # 날짜마다
        title_list = []
        for start in [1, 11, 21, 31]: # start 지정마다 10개 기사 추출 - start=1: 1~10, start=11: 11~20 ...
                date1 = date.replace('.', '') # 형태 변환 (2023.06.01 -> 20230601)
                url =  "https://search.naver.com/search.naver?where=news"\
                        + f"&query={encQuery}"\
                        + f"&ds={date}&de={date}"\
                        + f"&nso=so%3Ar%2Cp%3Afrom{date1}to{date1}"\
                        + f"&start={start}"
                        
                time.sleep(1)
                
                response = requests.get(url, headers=headers).content
                # response.raise_for_status()
                soup = BeautifulSoup(response, "html.parser")
                
                # 제목 추출
                titles = [title['title'] for title in soup.find_all('a', attrs={'class':'news_tit'})]

                # 모든 제목을 리스트에 추가
                title_list.extend(titles)

                # print(len(title_list))

        # 새로운 데이터 프레임 생성 및 기존 데이터 프레임과 결합
        temp_df = pd.DataFrame({
                'Title': title_list,
                'Date': [date] * len(title_list)  # 모든 제목에 대해 동일한 날짜 지정
        })
        news_df = pd.concat([news_df, temp_df], ignore_index=True)

print("----끝----") 

100%|██████████| 241/241 [40:12<00:00, 10.01s/it]

----끝----


In [6]:
news_df

,Title,Date
0,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",2023.06.01
1,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],2023.06.01
2,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",2023.06.01
3,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,2023.06.01
4,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',2023.06.01
...,...,...
9627,"[MD포토] 이희준, '완벽한 핸섬가이'",2024.05.27
9628,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,2024.05.27
9629,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",2024.05.27
9630,"[MD포토] 공승연, '정말 재밌어요~'",2024.05.27


In [7]:
# 날짜별로 기사가 40개씩 제대로 모아졌나 확인
groupby_news_df = news_df.groupby(['Date'])[['Title']].count()
groupby_news_df['Title'].value_counts() # 읭?

Title
40    239
39      1
33      1
Name: count, dtype: int64

In [8]:
# 40개가 아닌 날짜를 확인
groupby_news_df[groupby_news_df['Title'] != 40] # 2023.07.27 39개, 2024.02.08 33개
# url 확인해보니까 관련도 기준 기사가 각 39개, 33개였음 / 최대한으로 긁어온 거 확인

,Title
Date,
2023.07.27,39
2024.02.08,33


In [9]:
# 기사 제목이 중복되는 행 보기
news_df[news_df.duplicated(subset=['Title'], keep=False)]

# 확인해보니 같은 언론사 -> 무의미하다고 생각하여 삭제

,Title,Date
86,진해 경제자유구역 두동지구에 첨단 물류센터 ‘두동 하이브 스퀘어’ 신축,2023.06.05
93,진해 경제자유구역 두동지구에 첨단 물류센터 ‘두동 하이브 스퀘어’ 신축,2023.06.05
126,"[단독]좌석랜덤, 환불땐 돈내라?…세븐틴 공연 무더기 신고 당했다",2023.06.07
127,"두나무, 新생존전략..NFT+AI+ESG",2023.06.07
128,"""한자리 20만 원"" 천정부지 콘서트 티켓값...'등골 브레이커' 만든다",2023.06.07
...,...,...
9244,"'경영권 분쟁' 하이브, 20만원 깨졌다…'어닝쇼크' JYP 6만원 '간당'[핫종목]",2024.05.13
9245,"[ET라씨로] 하이브 주가, 20만원선 무너졌다",2024.05.13
9255,방시혁 ‘플랫폼 제국’ 꿈꿨지만…,2024.05.13
9275,"하이브 망신 넘어 K팝 공든 탑 위협…그만해, 이러다 다 죽어[박동흠의 생활 속 회...",2024.05.14


In [10]:
# 기사 제목 기준 중복되는 행 삭제
news_df = news_df.drop_duplicates(subset=['Title'])

In [11]:
news_df

,Title,Date
0,"하이브, 새 걸그룹 선보인다…데뷔 서바이벌 프로그램 론칭",2023.06.01
1,BTS '단체활동 중단' 미리 알고…하이브 직원들 주식매도 적발 [1분뉴스],2023.06.01
2,"미공개정보 이용한 하이브 직원 후회막심?...SM사태 이후 하이브 급등, 카카오 하락",2023.06.01
3,다 오를 때 혼자 부진했는데…'엑소'와의 갈등에 또 다시 악재 맞은 SM,2023.06.01
4,“하이브만이 아니었네”…올해 주식 불공정거래 절반이 '미공개정보 이용',2023.06.01
...,...,...
9627,"[MD포토] 이희준, '완벽한 핸섬가이'",2024.05.27
9628,뉴진스 앨범 첫날 81만장…민희진 해임 가처분 이번주 결론,2024.05.27
9629,"민희진 공격 vs 아일릿 보호…‘뉴진스 표절’ 둔 고소전, 어떻게 봐야할까 [D:이슈]",2024.05.27
9630,"[MD포토] 공승연, '정말 재밌어요~'",2024.05.27


In [13]:
news_df.to_csv('hybe_news.csv', index=False)